## Truncate generation times distribution
Here, we will truncate the generation times distribution,
to avoid the need always to make the calculations
over the full simulation period at each time step.
Considering all time points prior to the 
time of interest is unlikely to be desirable
for longer (including real-world) analyses.
All functions except for `renew` and 
`get_gamma_densities_from_params`
are unchanged from the previous notebook, 
other than feeding through one additional argument
for the number of generation times to use.

Note that I did try out the approach of
calculating the truncation time for the generation
time distribution within the model function,
such that it could be determined by the 
distribution being used. However, this resulted 
in the optimisations all going completely haywire.
Although I'm not quite sure why this happened,
this version is probably fine for our purposes
and allowing for adaptive truncations is probably
unnecessary.

In [ ]:
from typing import Dict, List
import numpy as np
import pandas as pd
from scipy.optimize import minimize

from distributions import get_gamma_params_from_mean_sd, get_gamma_densities_from_params
from process import get_interp_vals_over_model_time
from renew import renew_trunc_gen
from outputs import Outputs, plot_output_fit

In [ ]:
def model_func(gen_time_mean: float, gen_time_sd: float, process_req: List[float], pop: int, seed: int, n_times: int, gen_times_end: int) -> tuple:
    """The other epidemiological aspects of the model.
    """
    gen_time_densities = get_gamma_densities_from_params(gen_time_mean, gen_time_sd, n_times)
    process_vals = get_interp_vals_over_model_time(process_req, n_times)
    process_vals_exp = np.exp(np.array(process_vals))
    model_result = renew_trunc_gen(gen_time_densities, process_vals_exp, pop, seed, n_times, gen_times_end)
    return model_result, process_vals_exp

def calib_func(parameters: List[float], pop: int, seed: int, n_times: int, targets: dict, ) -> float:
    """Get the loss function from the model.
    """
    gen_time_mean, gen_time_sd, *process_req = parameters
    incidence = model_func(gen_time_mean, gen_time_sd, process_req, pop, seed, n_times, gen_times_end)[0][0]
    return sum([(incidence[t] - d) ** 2 for t, d in targets.items()])

### Optimisation with above model adaptations
Simple, arbitrary parameters and targets

In [ ]:
n_times = 40
infectious_seed = 1.0
population = 100.0
dummy_data = pd.Series(
    {
        5: 1.0,
        10: 1.0,
        15: 1.5,
        25: 4.2,
        30: 3.8,
        35: 2.1,
    },
)
gen_time_mean = 5.5
gen_time_sd = 1.7
max_gen_mean = 6.0
max_gen_sd = 2.0

# Calculate the longest distribution we could be interested in outside of the model function,
# otherwise if this is allowed to adapt to the distribution, things seem to go haywire
long_gen_densities = get_gamma_densities_from_params(max_gen_mean, max_gen_sd, n_times)
gen_times_end = np.argmax(long_gen_densities.cumsum() > 0.999)

param_bounds = [[5.0, max_gen_mean]] + [[1.5, max_gen_sd]] + [[-1000.0, np.log(10.0)]] * 4
result = minimize(calib_func, [5.0, 1.5] + [np.log(2.0)] * 4, method='Nelder-Mead', args=(population, infectious_seed, n_times, dummy_data), bounds=param_bounds)
model_result, process_vals = model_func(result.x[0], result.x[1], result.x[2:], population, infectious_seed, n_times, gen_times_end)
optimised, suscept, r_t = model_result
plot_output_fit(dummy_data.to_dict(), model_result, process_vals, n_times)